In [1]:
using LinearAlgebra

include("../src/tebd.jl")
using .TimeEvolvingBlockDecimation
using .TimeEvolvingBlockDecimation.MatrixProductState

Let's initialize a 12 qubit state in the $|0\cdots0\rangle$ state. Normally we would work with the representation 
$$|\psi\rangle=\left(\begin{matrix}
    1\\
    0\\
    0\\
    \vdots\\
    0
\end{matrix}\right)$$
with $$|\psi\rangle\in C^{\left(2^{12}\right)}$$
But instead we'll expand the tensor products implicit in $$|00\cdots0\rangle=|0\rangle\otimes|0\rangle\otimes\cdots$$ explicitly by writing out the full tensor. In other words we will instead use
    $$|\psi\rangle\in C^{2\times2\times\cdots\times2}$$
    with
    $$(|\psi\rangle)_{00\cdots0}=1.$$

In [2]:
sites = 12
ψ = zeros((2 for _=1:sites)...)
ψ[(1 for _=1:sites)...] = 1;

Now we convert $|\psi\rangle$ into an MPS:

In [3]:
ψ_mps = mps(ψ);

With this representation, it is much less constly to apply local operators to the state $|\psi\rangle$. Instead of calculating the matrix exponential $e^{-i\hat{H}t}$ using the full hamiltonian $\hat{H}$, we can apply one- or two-local operators at each site in the chain, reinforce the maximum allowed bond dimension, and repeat for each site or pair of sites that is included in $\hat{H}$.

In [4]:
# Define Pauli basis and tensor product shorthand.
# Use t=π/6 for this example.
⊗ = kron
I = [1 0; 0 1]
σ_x = [0 1; 1 0]
σ_y = [0 -1im; 1im 0]
σ_z = [1 0; 0 -1]

t = π / 6;

First, let's calculate the time-evolved state
$$|\psi(t)\rangle = exp\left\{-i\hat{H}t\right\}|\psi\rangle,$$
where the Hamiltonian
$$\hat{H}=\sum_i\sigma_x^{(i)}$$
is a time-independent sum of single-qubit operators. The extremely naive method to calculate $|\psi(t)\rangle$ is to first calculate the matrix exponential, then do a matrix-vector multiplication. Let's do it this way first:

In [5]:
# The not as fast way

# Helper function for calculating H. We only use local terms in this example.
function ising_matrix(sites::Integer, local_only::Bool = false)
        I = [
            1 0
            0 1
        ]
        identity_string = [I for _ = 1:sites]
        local_term = zeros(2^sites, 2^sites)
        interaction_term = zeros(2^sites, 2^sites)

        for i = 1:sites
            pauli_string = copy(identity_string)
            pauli_string[i] = σ_x
            local_term += reduce(kron, pauli_string)
        end

        for i = 1:sites-1
            pauli_string = copy(identity_string)
            pauli_string[i] = σ_z
            pauli_string[i+1] = σ_z
            interaction_term += reduce(kron, pauli_string)
        end
        if local_only
            return local_term
        end
        return interaction_term + local_term
end

# Only use local terms for H:
H = ising_matrix(sites, true)
ψ_vect = zeros(2^sites)
ψ_vect[1] = 1

# Naive method: do the matrix exponential followed by a matrix-vector multiplication.
@time ψ_res_vect = exp(H * t * -1im) * ψ_vect

 27.308956 seconds (6.68 M allocations: 4.726 GiB, 1.03% gc time, 6.94% compilation time)


4096-element Vector{ComplexF64}:
    0.17797851562499956 + 0.0im
                    0.0 - 0.10275594390606246im
                    0.0 - 0.10275594390606206im
   -0.05932617187500035 + 0.0im
                    0.0 - 0.10275594390606241im
   -0.05932617187500032 + 0.0im
   -0.05932617187500048 + 0.0im
                    0.0 + 0.034251981302021194im
                    0.0 - 0.10275594390606213im
  -0.059326171875000305 + 0.0im
  -0.059326171875000305 + 0.0im
                    0.0 + 0.03425198130202117im
  -0.059326171875000416 + 0.0im
                        ⋮
                    0.0 - 0.001268591900074864im
 -0.0007324218750000013 + 0.0im
 -0.0007324218750000011 + 0.0im
                    0.0 + 0.0004228639666916218im
                    0.0 - 0.001268591900074864im
 -0.0007324218750000008 + 0.0im
  -0.000732421875000001 + 0.0im
                    0.0 + 0.00042286396669162214im
  -0.000732421875000001 + 0.0im
                    0.0 + 0.000422863966691622im
                    

Thanks to the `@time` macro, we can see that this calculation takes about 20 seconds and uses a whole lot of memory. Pretty bad. Alternatively, let's use block evolution on a MPS representation of $|\psi\rangle$. With this method, we only do twelve $2\times2$ matrix-vector multiplications, and since our initial state was unentangled the result is exact. Where our initial state (or system Hamiltonian) more highly entangled (entangling) the MPS representation (block evolution) would do a poorer job approximating the full matrix multiplication.

In [8]:
# The fast way
local_only_ising = Hamiltonian(0 * (σ_z ⊗ σ_z), σ_x)
@time ψ_res = reshape(contract_mps(block_evolve(ψ_mps, local_only_ising, t * -1im)), 2^sites)

  0.007025 seconds (134.89 k allocations: 9.755 MiB)


4096-element Vector{ComplexF64}:
      0.1779785156250009 - 7.705139889271523e-35im
  -4.448564589214635e-35 - 0.10275594390606424im
  -1.689546579027104e-35 - 0.10275594390606424im
    -0.05932617187500025 + 9.754601722097095e-36im
   7.323680981525651e-35 - 0.10275594390606425im
    -0.05932617187500026 - 4.228329186142776e-35im
   -0.059326171875000264 - 1.605902076541324e-35im
  -9.271679961833121e-36 + 0.034251981302021395im
  -4.448564589214636e-35 - 0.10275594390606424im
    -0.05932617187500025 + 2.568379963090506e-35im
    -0.05932617187500025 + 9.754601722097097e-36im
  5.6318219300903436e-36 + 0.03425198130202139im
    -0.05932617187500027 - 4.2283291861427763e-35im
                         ⋮
   9.041581458673635e-37 - 0.0012685919000748663im
  -0.0007324218750000026 - 5.22015948906515e-37im
  -0.0007324218750000027 - 1.982595156223855e-37im
 -1.1446518471398905e-37 + 0.0004228639666916218im
  -5.492055048413126e-37 - 0.001268591900074866im
  -0.0007324218750000024 + 3.17083

To prove the two results are equal:

In [9]:
dot(ψ_res_vect, ψ_res)

1.0000000000000018 - 1.2379560085042535e-34im